<a href="https://colab.research.google.com/github/komo135/forex-prediction/blob/master/ipynb/rl/sac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [2]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(sac)
except:
  import sac

agent = sac.Agent(restore=not True, n=1, lr=1e-4)

/content/drive/My Drive


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
%cd /content

agent.run(train=True)

/content
action probability: buy=0.13541666666666666, sell=0.4166666666666667, hold=0.4479166666666667
epoch: 3000, total assets:102896.53794441969, growth_rate:1.0289653794441969

action probability: buy=0.11458333333333333, sell=0.3541666666666667, hold=0.53125
epoch: 3005, total assets:4594.957999231806, growth_rate:0.04594957999231806

action probability: buy=0.11458333333333333, sell=0.4583333333333333, hold=0.4270833333333333
epoch: 3010, total assets:3482.201079545045, growth_rate:0.03482201079545045

action probability: buy=0.13541666666666666, sell=0.4166666666666667, hold=0.4479166666666667
epoch: 3015, total assets:28182.921568994323, growth_rate:0.28182921568994324

action probability: buy=0.11458333333333333, sell=0.4270833333333333, hold=0.4583333333333333
epoch: 3020, total assets:39861.56284339479, growth_rate:0.39861562843394793

action probability: buy=0.11458333333333333, sell=0.4375, hold=0.4479166666666667
epoch: 3025, total assets:13131.143051866045, growth_rate:0

In [0]:
for _ in range(1):
  # agent.test(spread=10, pip_cost=1000, los_cut=150,test_data= not True)
  tree_idx, replay = agent.memory.sample(256)

  states = np.array([a[0][0] for a in replay])
  new_states = np.array([a[0][3] for a in replay], np.float32)
  actions = np.array([a[0][1] for a in replay]).reshape((-1, 2))
  rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

  ent_coef = tf.exp(agent.model.log_ent_coef)
  ################################################################################
  with tf.GradientTape() as p_tape:
      d, policy, logp_pi = agent.model.actor(states)
      q1_pi, q2_pi, _ = agent.model.critic([states, policy])
      # q1, q2, v = self.model.critic.predict_on_batch([states, actions])
      min_q_pi = tf.minimum(q1_pi, q2_pi)
      # p_loss = -tf.reduce_mean(q1_pi) - tf.reduce_mean(q2_pi)
      # p_loss = tf.reduce_mean(ent_coef * logp_pi * 100 - q1_pi)
      p_loss = tf.reduce_mean(ent_coef * logp_pi * 2 - (q1_pi + q2_pi))
      # p_loss = tf.reduce_mean(ent_coef * logp_pi - q2_pi)
  ################################################################################
  with tf.GradientTape() as v_tape:
      _, _, target_v = agent.target_model.critic([new_states, actions])
      q1, q2, v = agent.model.critic([states, actions])
      min_q_pi = tf.minimum(q1_pi, q2_pi)

      q_backup = rewards + agent.gamma * target_v

      q1_loss = 0.5 * tf.reduce_mean((q_backup - q1) ** 2)
      q2_loss = 0.5 * tf.reduce_mean((q_backup - q2) ** 2)

      v_backup = min_q_pi - ent_coef * logp_pi
      v_loss = 0.5 * tf.reduce_mean((v_backup - v) ** 2)

      v_loss += q1_loss + q2_loss
  ################################################################################
  with tf.GradientTape() as e_tape:
      e_loss = -tf.reduce_mean(agent.model.log_ent_coef * logp_pi + agent.model.target_entropy)
  ################################################################################

  ae = np.abs(np.array(q_backup) - min_q_pi.numpy()).reshape((-1,))
  # print(ae)
  # agent.memory.batch_update(tree_idx, ae)

  gradients = v_tape.gradient(v_loss, agent.model.critic.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -1.0, 1.0))
  #              for grad in gradients]
  # agent.v_opt.apply_gradients(zip(gradients, agent.model.critic.trainable_variables))

  p_gradients = p_tape.gradient(p_loss, agent.model.actor.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -1.0, 1.0))
  #               for grad in gradients]

  # agent.p_opt.apply_gradients(zip(gradients, agent.model.actor.trainable_variables))

  # gradients = e_tape.gradient(e_loss, agent.model.log_ent_coef)
  # gradients = (tf.clip_by_value(gradients, -1.0, 1.0))
  # agent.e_opt.apply_gradients([[gradients, agent.model.log_ent_coef]])

In [8]:
%%timeit
d, policy, logp_pi = agent.model.actor(states)

10 loops, best of 3: 20.8 ms per loop


In [11]:
%%timeit
d, policy, logp_pi = agent.model.actor.predict_on_batch(states)

100 loops, best of 3: 4.32 ms per loop


In [9]:
%%timeit
d, policy, logp_pi = agent.policy(states)

1000 loops, best of 3: 1.43 ms per loop


In [41]:
i = 89
print(q2_pi[i])
print(q2[i])
print(v[i])
print(q_backup[i])

tf.Tensor([-6.6349797], shape=(1,), dtype=float32)
tf.Tensor([-6.635057], shape=(1,), dtype=float32)
tf.Tensor([-0.06495996], shape=(1,), dtype=float32)
tf.Tensor([-1.6617409], shape=(1,), dtype=float32)


In [0]:
np.array([int((i1 - i2) * 10 ** 2) / (10 ** 2) for i1, i2 in zip(q2,q2_pi)]).reshape((-1,)).astype(np.float64)

In [0]:
q1

In [0]:
import sys

import numpy as np
import pandas as pd
import ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


def gen_data(file_path="gbpjpy15.csv"):
    try:
        print("load file")
        df = pd.read_csv(file_path)
    except:
        print("Use 'python gen_data.py 'file_path''")
        return

    df["Close1"] = df["Close"] * 100

    ma = np.array(ta.trend.ema(df["Close1"], 14) - ta.trend.ema(df["Close1"], 7)).reshape((-1,1))
    macd = np.array(ta.trend.macd_diff(df["Close1"])).reshape((-1,1))
    rsi = np.array(ta.momentum.rsi(df["Close"]) - ta.momentum.rsi(df["Close"], 7)).reshape((-1,1))
    stoch = np.array(ta.momentum.stoch_signal(df["High"], df["Low"], df["Close"]) - ta.momentum.stoch(df["High"], df["Low"], df["Close"])).reshape((-1,1))

    x = np.concatenate([ma, macd, rsi, stoch], -1)

    y = np.array(df[["Open"]])
    atr = np.array(ta.volatility.average_true_range(df["High"], df["Low"], df["Close"]))
    high = np.array(df[["High"]])
    low = np.array(df[["Low"]])

    print("gen time series data")
    x = x[100:]
    y = y[100:]

    window_size = 130
    time_x = []
    time_y = []

    for i in range(len(y)-window_size):
      time_x.append(x[i:i+window_size])
      i += window_size
      time_y.append(y[i])
    
    x = np.array(time_x).reshape((-1, window_size, x.shape[-1]))
    y = np.array(time_y).reshape((-1, y.shape[-1]))

    atr = atr[-len(y):].reshape((-1, 1))
    scale_atr = atr
    high = high[-len(y):].reshape((-1, 1))
    low = low[-len(y):].reshape((-1, 1))

    x = (x * 10 ** 5).astype(np.int32) * (10 ** -5)

    np.save("x", x)
    np.save("target", np.array([y, atr, scale_atr, high, low]))

    print("done\n")


In [0]:
%cd /content/drive/My Drive

gen_data()

In [0]:
h = np.random.randint(agent.x.shape[0] - agent.x.shape[0] * 0.2 - agent.step_size)

df = agent.x[h:h + agent.step_size]
trend = agent.y[h:h + agent.step_size]
atr = agent.atr[h:h + agent.step_size]
scale_atr = agent.scale_atr[h:h + agent.step_size]
high = agent.high[h:h + agent.step_size]
low = agent.low[h:h + agent.step_size]

memory = deque()

In [13]:
1 - (0.1 + (1 - 0.1) * (np.exp(-0.00005 * 20000)))

0.5689085029457018